In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

**Initiate the model**

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

**Sentiment analysis for simple sentences**

In [5]:
tokens = tokenizer.encode("that's a great game but it lags a lot when there are a lot of players", return_tensors='pt')

In [6]:
result = model(tokens)

In [7]:
#the highest number is the rating out of this list of tensor :
result.logits

tensor([[-1.5472,  0.2167,  1.9390,  1.0027, -1.3699]],
       grad_fn=<AddmmBackward0>)

In [8]:
#to simplify the rating and make it between 1 and 5:
int(torch.argmax(result.logits))+1

3

**scrapping all the reviews of a service on yelp.com**

In [9]:
r = requests.get('https://www.yelp.com/biz/burger-king-san-francisco-10')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [10]:
reviews

["First of all I would like to say, Burger King Really great to make a delicious burger. Staff's are very gentle, a nice staff. But my wife loved the Ambiance place.",
 'Worst burger king, they would not open the bathrooms even tho they\'re working because they laughed when i told the manager "so u just don want to clean it" meat was uncooked and vegetables were old. i do not recommend this burger king.',
 "The place has plenty of seats. Can't miss the little restaurant as it is literally just next to a good will store and a AMC theater just one street above. The floor is very dirty and certain section of the floor are very greasy thus be careful when walking. Lets talk about their \xa0spicy ch'king sandwich. This sandwich title is very misleading. Got the sandwich and the chicken came in mildly crunchy with lettuce, tomato, mayonnaise, and sweet sauce rather than a spicy red sauce. Chicken is overcooked resulting in a very dry bite, but the sauce and mayonnaise made it a bit easier to

**Load Reviews into DataFrame and Score**

In [11]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [12]:
df

,review
0,"First of all I would like to say, Burger King ..."
1,"Worst burger king, they would not open the bat..."
2,The place has plenty of seats. Can't miss the ...
3,"Horrible customer serviceTried the app, 20 min..."
4,I ordered today and was very unsatisfied with ...
5,Food was good and food prep area looked clean....
6,This is yet another location where the service...
7,Do not eat here. .. they gave my food away and...
8,I give this place no stars horrible service pl...
9,I'm warning you: do not enter this establishme...


In [13]:
#function to rate texts
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [14]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [15]:
df

,review,sentiment
0,"First of all I would like to say, Burger King ...",4
1,"Worst burger king, they would not open the bat...",1
2,The place has plenty of seats. Can't miss the ...,2
3,"Horrible customer serviceTried the app, 20 min...",1
4,I ordered today and was very unsatisfied with ...,1
5,Food was good and food prep area looked clean....,2
6,This is yet another location where the service...,4
7,Do not eat here. .. they gave my food away and...,1
8,I give this place no stars horrible service pl...,1
9,I'm warning you: do not enter this establishme...,1
